# Introduction

In this notebook, we will create a convolutional neural network (CNN) to classify images. We will use the built-in dataset called Caltech 101, which consists of colored images belonging to 101 classes.

But before we begin, let's define CNN in a simpler way :books:.

> " In deep learning, a convolutional neural network (CNN) is a class of artificial neural network most commonly applied to analyze visual imagery. CNNs use a mathematical operation called convolution in place of general matrix multiplication in at least one of their layers. They are specifically designed to process pixel data and are used in image recognition and processing."[Wikipedia](https://en.wikipedia.org/wiki/Convolutional_neural_network).

The main idea behind CNN is to apply filters to the input data in order to extract relevant features through convolution. Additionally, CNNs utilize pooling to reduce dimensionality, making the data suitable for fully connected layers in the neural network.


![CNN](../images/CNN.png)


If you are unfamiliar with CNNs, there is a YouTube playlist that explains the concepts of deep learning and neural networks presented by Josh Starmer. You can find it [Here](https://youtube.com/playlist?list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1).

#### Table of contents :

- **Collecting & Getting Data**.
- **Data Analysis**.
- **Data Transformation / Augmentation**.
- **Building a CNN model**.
- **Evaluate Mode**.
- **Save & Load Model**.


We need to introduce the modules and libraries that we will use and need before writing the code.

PyTorch Modules|                                  Definition
-----|----
torch.nn | it provides classes and functions for building and training neural networks by offering a high-level API for defining layers, activation functions, loss functions, optimizers, and other components.
torchvision.datasets | it provides pre-built datasets commonly used in computer vision tasks, making it easier to load and preprocess data for tasks such as image classification, object detection, and semantic segmentation.
torchvision.transforms | it provides a set of common image transformations, such as resizing, cropping, flipping, and normalization, which can be applied to images or tensors to augment and preprocess the data before feeding it into a neural network for tasks like computer vision.
torch.utils.data.DataLoader| it provides an efficient way to load and iterate over datasets. It allows you to create an iterable data loader object that automatically handles tasks like batching, shuffling, and parallel data loading, making it easier to train neural networks on large datasets.

Import Modules 

In [1]:
import torch
import torch.nn as nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision import transforms

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")


PyTorch version: 2.0.0
torchvision version: 0.15.0


### 1. Collect or Getting Data

In [2]:
print(' Hello , World ')

 Hello , World 
